TBD:
1. Revisit the raw expected value of all the crates from R2, and see what trends emerged. Did traders make the most naive choices or did they put more thought into it? Figure out real profits, try and figure out which option was most profitable.
2. For extra information, compute Nash equilibrium, and produce a distribution of optimal choices for all the crates.
3. Estimate the distribution of choices other teams will make, and run simulations to model choices other teams make.

Starting with task 1.

R2 EVs (most naive):

[10, 13.33333, 12.33333, 17, 9, 15.5, 12.5, 10, 18.25, 11.12]

Crate 1: 10
Crate 2: 13.33333 (80/6)
Crate 3: 12.33333 (37/3)
Crate 4: 17
Crate 5: 9
Crate 6: 15.5 (31/2)
Crate 7: 12.5 (50/4)
Crate 8: 10
Crate 9: 18.25 (73/4)
Crate 10: 11.12 (89/8)



Nash equilibrium distribution:
x = [0.018370221327861633, 0.16696177062331183, 0.07496981891332749, 0.03822937625685432, 0.1553319919548541, 0.06794768611594418, 0.10185110663970436, 0.0367404426560982, 0.16710261569210677, 0.17249496981993712]


R2 trader choices.

Crate 1: 0.0998 (<)
Crate 2: 0.18178 (>)
Crate 3: 0.05118 (<<)
Crate 4: 0.07539 (>>)
Crate 5: 0.11807 (<<)
Crate 6: 0.06987 (>)
Crate 7: 0.08516 (<)
Crate 8: 0.01614 (<<)
Crate 9: 0.24060 (>>)
Crate 10: 0.15185 (<)

What were the most profitable crates?

Crate 1: 5
Crate 2: 3.33
Crate 3: 4.5
Crate 4: 2
Crate 5: 4.12
Crate 6: 3.4444
Crate 7: 4
Crate 8: 7.69
Crate 9: 2.6
Crate 10: 3.869

In [3]:
arr = [10, 13.33333, 12.33333, 17, 9, 15.5, 12.5, 10, 18.25, 11.12]
print("AVERAGE EV FOR CRATE IN R2: ", sum(arr) / 10)

AVERAGE EV FOR CRATE IN R2:  12.903666000000001


Clear pattern: The crates with the low raw expected values are underselected, and the crates with the high raw expected values are overselected. Raw expected value means (multiplier / inhabitants), not accounting for teams' selections. The baseline for 'selection' or 'underselection' is the Nash equilibriumn. I think this underselection/overselection effect is accentuated as the raw expected values for a crate deviate from the mean.

This means that we should assume there's a small sect of the traders that are naive, and design our simulations accordingly.

Working on task 2. What's the nash equilibrium (or the baseline distribution of teams' choices)?

In [8]:
class Box:
  def __init__(self, label, multiplier, contestants):
    self.label = label
    self.multiplier = multiplier
    self.contestants = contestants
    self.raw_ev = multiplier / contestants
    self.expected_team_proportion = 0.05
    self.predicted_proportion = 0
    self.predicted_ev = 0
  def __str__(self):
    return "Box {l}. Multiplier: {m}, Contestants: {c}, Raw EV: {rev}, Nash Proportion: {prop}, Predicted Proportion: {p}, Predicted Profit: {pev}".format(l = self.label, m = self.multiplier, c = self.contestants, rev = self.raw_ev, prop = self.expected_team_proportion, p = self.predicted_proportion, pev = self.predicted_ev)


raw_boxes = [('1A', 80, 6), ('2A', 50, 4), ('3A', 83, 7), ('4A', 31, 2), ('5A', 60, 4), ('1B', 89, 8), ('2B', 10, 1), ('3B', 37, 3), ('4B', 70, 4), ('5B', 90, 10), ('1C', 17, 1), ('2C', 40, 3), ('3C', 73, 4), ('4C', 100, 15), ('5C', 20, 2), ('1D', 41, 3), ('2D', 79, 5), ('3D', 23, 2), ('4D', 47, 3), ('5D', 30, 2)]

boxes = [Box(raw_box[0], raw_box[1], raw_box[2]) for raw_box in raw_boxes]

from scipy.optimize import fsolve
import numpy as np

def equations(vars):
  x = vars[:20]
  nash = vars[20]

  eqs = [((nash*(boxes[i].contestants + x[i]*100)) - boxes[i].multiplier) for i in range(20)]

  eqs.append(sum(x) - 1)

  return eqs

initial_guess = [0.1] * 20 + [1.0]

solution = fsolve(equations, initial_guess)

# solution_dict = {f"x_{(i+1)}": solution[i] for i in range(20)}

# Add to boxes
for i in range(20):
  boxes[i].expected_team_proportion = solution[i]

print("NASH: ", solution[20])

# Compute the expected distribution
# What are the categories of people?
# DUMBASSES: pick (two) suitcase(s) with highest raw EV (4% on #2, 6% on #1)
# DUMBASSES+: pick (two) suitcase(s) with lowest raw EV (2.5% on -1, 1.5% on -2)
# SMART: pick two suitcases - one middle for risk, and one high as a hedge, to reduce sensitivity (1.25% on middle, 1.25% mid-high)
# SMART+: pick two suitcases - one middle for risk, and one low as a hedge, to increase profit (0.5% on middle, 0.5% on mid-low)
# RANDOM: pick suitcase at random

# Implications:
# DUMBASSES -> bump up suitcases with two highest raw EVs (and subtract the value you bumped up from every other suitcase)
# DUMBASSES+ -> bump the lowest EV picks.
# NASHERS: bump up the middle, bump up mid-high suitcases.
# NASHERS+: bump up the middle, bump up the low as a hedge

# Set the default predicted proportion
for box in boxes:
  box.predicted_proportion = box.expected_team_proportion

# DUMBASSES
boxes.sort(key=lambda box: box.raw_ev, reverse=True)
boxes[0].predicted_proportion += 0.06
boxes[1].predicted_proportion += 0.04
for i in range(2, 20):
  boxes[i].predicted_proportion -= (0.1 / 18)

# DUMBASSES+
boxes.sort(key=lambda box: box.raw_ev, reverse=False)
boxes[0].predicted_proportion += 0.025
boxes[1].predicted_proportion += 0.015
for i in range(2, 20):
  boxes[i].predicted_proportion -= (0.04 / 18)

# SMART / SMART+
boxes.sort(key=lambda box: box.raw_ev, reverse=False)
for box in boxes[7:13]:
  box.predicted_proportion += (0.0175 / 6)
for box in boxes[:7] + boxes[14:]:
  box.predicted_proportion -= (0.0175 / 14)
# mid-to-high
for box in boxes[13:18]:
  box.predicted_proportion += (0.0125 / 5)
for box in boxes[:13] + boxes[18:]:
  box.predicted_proportion -= (0.0125 / 15)
# mid-to-low
for box in boxes[2:7]:
  box.predicted_proportion += (0.005 / 5)
for box in boxes[:2] + boxes[7:]:
  box.predicted_proportion -= (0.005 / 15)

# boxes.sort(key=lambda box:box.predicted_proportion, reverse=True)

# Predicted profits
for box in boxes:
  box.predicted_ev = box.multiplier / (box.contestants + 100*box.predicted_proportion)

boxes.sort(key=lambda box:box.expected_team_proportion, reverse=True)

for box in boxes:
  print(box)







NASH:  5.661375661382135
Box 2D. Multiplier: 79, Contestants: 5, Raw EV: 15.8, Nash Proportion: 0.08954205607332895, Predicted Proportion: 0.08268094496221784, Predicted Profit: 5.9541330537324715
Box 3C. Multiplier: 73, Contestants: 4, Raw EV: 18.25, Nash Proportion: 0.08894392523228832, Predicted Proportion: 0.14430503634339947, Predicted Profit: 3.96082502400995
Box 4B. Multiplier: 70, Contestants: 4, Raw EV: 17.5, Nash Proportion: 0.08364485981191709, Predicted Proportion: 0.11900597092302818, Predicted Profit: 4.4023504019157675
Box 1A. Multiplier: 80, Contestants: 6, Raw EV: 13.333333333333334, Nash Proportion: 0.08130841121374627, Predicted Proportion: 0.07528063343596848, Predicted Profit: 5.9136328658503725
Box 1B. Multiplier: 89, Contestants: 8, Raw EV: 11.125, Nash Proportion: 0.07720560747545085, Predicted Proportion: 0.06834449636433974, Predicted Profit: 5.999548495645744
Box 3A. Multiplier: 83, Contestants: 7, Raw EV: 11.857142857142858, Nash Proportion: 0.07660747663441

2B is the risky option. It's risky because it is the worst crate. It's one of the worst according to raw EV. And, it's the least selected by the baseline Nash equilibrium distribution. It's bold to pick the WORST crate, but I kinda like it, it's ballsy

3A is the safe option. It has a high multiplier, so it's relatively insensitive to changes in teams choosing it, and according to my model, it has an expected profit of 60k.

There are a couple of other crates to consider for the risky option. There's 3D and there's 5C.